### Importing the data and the functions

In [ ]:
using DrugResponseModel

# population data, g2 data, g1 data, initial number of cells in g1, initial number of cells in g2
# lapatinib
conc_l, pop_l, g2_l, g1_l, g2_0_l, g1_0_l = setup_data("lapatinib");

### A) cartoon 

In [ ]:
# It is plotA.svg 

### B) ODE fitting of Lapatinib data

In [ ]:
# initial values
p = [8.870525324, 8.492087169, 0.43447323, 7.67847790]

# setting lowest delay for tau1 to be half an hour and for tau2 to be 3 hours.
low = 0.0001*ones(4)
upp = 0.1*ones(4)
params_ode = zeros(4, 8)

# ODE optimization and estimation of the parameters
for i in 1:8
    params_ode[:, i] = ODEoptimizer(low, upp, p, i, g1_l, g2_l, g1_0_l, g2_0_l)
end

In [ ]:
ODEplot_all(params_ode, g1_l, g2_l, g1_0_l, g2_0_l, pop_l)

### F) DDE fitting for lapatinib

In [ ]:
# initial guess
initial_guess  = [0.02798, 0.025502, 15.3481, 15.2881, 0.001, 0.001]

# bounds 
lower_bnd = [-6.0, -6.0, 1.0, 1.0, -10.0, -10.0]
upper_bnd = [0.0, 0.0, 6.0, 6.0, 0.0, 0.0]

# max number of steps
maxSteps = 10000
parameters = zeros(6,8)
for j in 1:8
    println("#################### trial number $j #########################")
    best_fit, parameters[:,j] = optimization(g1_l, g2_l, g1_0_l, g2_0_l, initial_guess, j, lower_bnd, upper_bnd, maxSteps)
    println(best_fit)
end

In [ ]:
plot_all(parameters, pop_l, g2_l, g1_l, g2_0_l, g1_0_l)

### G) DDE parameters for lapatinib

In [ ]:
plot_parameters(conc_l, parameters)

### H) DDE Hill fitting 

In [ ]:
#  [EC50, b_steepness, alpha_min, alpha_max, beta_min, beta_max, tau1_mean, tau1_max, tau2_min, tau2_max, gamma1_max, gamma2_max]
guess = [125.0, 0.4, 0.007, 0.005, 0.007, 0.005, 39.0, 30.0, 11.0, 20.0, 0.003, 0.02]
# max num of steps
num_steps=500

# optimize and return the estimated hill parameters
best_fit, params = optimize_hill(guess, conc_l, g1_l, g2_l, g1_0_l, g2_0_l, num_steps)

# easy way to convert the hill parameters to DDE parameters
p = getDDEparams(params, conc_l)

In [ ]:
plot_all(p, pop_l, g2_l, g1_l, g2_0_l, g1_0_l)

### I) DDE Hill Parameters

In [ ]:
plot_parameters(conc_l, p)

### The phase portrait of G1 and G2 data

In [ ]:
# lapatinib
labels = ["control", "conc. 5nM", "conc. 10nM", "conc. 25nM", "conc. 50nM", "conc. 100nM", "conc. 250nM", "conc. 500nM"]
xlabel = "G1 numbers"
ylabel = "G2 numbers"
ymax = 40
correlationPlot(g1_l, g2_l, labels, xlabel, ylabel, ymax)